<font size="7">Use this program to start the model</font>

<font size="5">1. Traverse the input folder and all subfolder, transfer all databases to a folder with csv files(tables)</font>

In [1]:
import magic
import Database_to_CSV    #import Database_to_CSV.py
import file_type_identify        #import is_Database.py
import os
import preprocess_wordfilter    #import preprocess_wordfilter.py
import langchain_chatgpt       #import langchain_chatgpt.py
import sameline_value          #import sameline_value.py
import datetime
import csv
import ast
import pandas as pd
import xml.etree.ElementTree as ET
# csv.field_size_limit(500 * 1024 * 1024)
csv.field_size_limit(1 * 1024 * 1024 * 1024)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import tiktoken
from langchain.text_splitter import TokenTextSplitter
import evidence_graph 
import xml_file_preprocess
import combine_csv
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 
# os.environ["OPENAI_API_KEY"] = ""



True
True
True


In [2]:
def preprocess_folder(folder_path,output_folder):
    # 遍历文件夹下的所有文件和子文件夹
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            # 检查文件是否为数据库文件
            print(f"Begin to analyze this file: {file_path}")
            print(f"root is {root}")
            print(f"file is {file}")
            #判断该文件是否以.db-wal结尾，如果是，则跳过，因为这个时候这个文件可能已经被删除了
            if file.endswith("-wal") or file.endswith("-shm"):
                continue          
            if file_type_identify.file_type_identify(file_path) == "SQLite database":
                # 调用导出函数
                Database_to_CSV.export_database_to_csv(file_path,output_folder)
                print("Transfer success")
                print(str(file)+"is database file")
                print(f"Transer success file path is: {file_path}")
            else:
                print("Transfer incomplete")
                print(str(file)+" is not database file")
            

In [ ]:
# input_folder_rawfiles="/home/hzhou/llama2/Android10_Image/10 Sample apps"
input_folder_rawfiles="/home/hzhou/llama2/Android10_Image/Apps in the Document"
output_folder = "/home/hzhou/llama2/Database_to_CSV/Database_to_CSV_Apps_in_the_Document"
preprocess_folder(input_folder_rawfiles,output_folder)

<font size="5">2. Use regular expression preprocess all csv file and combine all lines that may contain evidence into a single csv file </font>

In [ ]:
input_folder_csvfiles="/home/hzhou/llama2/Database_to_CSV/Database_to_CSV_Apps_in_the_Document"
output_after_filter_csv="/home/hzhou/llama2/llama/after_filter/Apps_in_the_Document/db_after_filter_Apps_in_the_Document.csv"

preprocess_wordfilter.csv_extract_lines_with_keywords(input_folder_csvfiles,output_after_filter_csv)  

In [ ]:
#second check match word whether name
csv_input_file="/home/hzhou/llama2/llama/after_filter/experiment_apps/after_filter_experiment_apps.csv"
csv_output_file="/home/hzhou/llama2/llama/after_filter/experiment_apps/after_filter_second_check_experiment_apps.csv"
langchain_chatgpt.analyze_csv_file_check_name(csv_input_file, csv_output_file)



In [6]:
#Analyze after_second_check csv file, if the evidence is name and second check is "Yes", combine them into a new csv file
csv_input_file="/home/hzhou/llama2/llama/after_filter/experiment_apps/after_filter_second_check_experiment_apps.csv"
csv_output_file="/home/hzhou/llama2/llama/after_filter/experiment_apps/wait_for_chatgpt_analyze_experiment_apps.csv"
langchain_chatgpt.combine_into_csv_file(csv_input_file, csv_output_file)


In [ ]:
#xml file preprocess
input_folder_rawfiles = "/home/hzhou/llama2/Android10_Image/Apps in the Document"
output_csv = '/home/hzhou/llama2/llama/after_filter/Apps_in_the_Document/xml_wait_for_chatgpt_analyze_App_in_the_Document.csv'
xml_file_preprocess.traverse_and_save_xml(input_folder_rawfiles, output_csv)

In [6]:
#Combine the database preprocessed file and xml preprocessed file together
database_wait_for_chatgpt_analyze_file= "/home/hzhou/llama2/llama/after_filter/Apps_in_the_Document/db_after_filter_Apps_in_the_Document.csv"
xml_wait_for_chatgpt_analyze_file="/home/hzhou/llama2/llama/after_filter/Apps_in_the_Document/xml_wait_for_chatgpt_analyze_App_in_the_Document.csv"
combine_wait_for_chatgpt_analyze_file="/home/hzhou/llama2/llama/after_filter/Apps_in_the_Document/combine_wait_for_chatgpt_analyze_App_in_the_Document.csv"

combine_csv.combine_preprocessed_data(database_wait_for_chatgpt_analyze_file,xml_wait_for_chatgpt_analyze_file,combine_wait_for_chatgpt_analyze_file)


<font size="5">3. Use chatgpt and langchain to analyze after_filter.csv file</font>

In [ ]:
#Use gpt model analyze "name","address","phone number" or "email" from preprocessed csv files 
request_evidence_type="name"

wait_for_chatgpt_analyze_csv="/home/hzhou/llama2/llama/after_filter/Apps_in_the_Document/combine_wait_for_chatgpt_analyze_App_in_the_Document.csv"
output_after_chatgpt_analyze_csv= "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/name_combinefiles_after_chatgpt_analyze_App_in_the_Document.csv"

overall_chain=langchain_chatgpt.Langchain_chatgpt_analyze(request_evidence_type)
langchain_chatgpt.analyze_csv_file(wait_for_chatgpt_analyze_csv, output_after_chatgpt_analyze_csv,overall_chain)

In [3]:
# combine the result file of evidence "name", "address", "phone number" and "email", maybe include "username"
combine_used_path= "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/Combined used files"
file_after_combine= "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/6evidence_combinefiles_after_chatgpt_analyze_App_in_the_Document.csv"

combine_csv.combine_llm_result(combine_used_path,file_after_combine)


In [ ]:
#Use gpt model analyze other evidence from those line have already be detect evidence 
#This part use to analyze "username"
request_evidence_type="username"
combine_result_file_after_first_analyze = "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/4evidence_combinefiles_after_chatgpt_analyze_App_in_the_Document.csv"
output_after_chatgpt_analyze_csv_username = "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/username_after_chatgpt_analyze_App_in_the_Document.csv"

overall_chain=langchain_chatgpt.Langchain_chatgpt_analyze(request_evidence_type)
langchain_chatgpt.analyze_csv_file_condtion_username(combine_result_file_after_first_analyze, output_after_chatgpt_analyze_csv_username,overall_chain)



In [ ]:
# #This part use chatgpt to analyze "appname" based on the stroage_path
request_evidence_type="appname"
combine_result_file_after_first_analyze = "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/5evidence_combinefiles_after_chatgpt_analyze_App_in_the_Document.csv"
output_after_chatgpt_analyze_csv_username = "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/appname_after_chatgpt_analyze_App_in_the_Document.csv"

overall_chain=langchain_chatgpt.Langchain_chatgpt_analyze(request_evidence_type)
langchain_chatgpt.analyze_csv_file_gpt_appname(combine_result_file_after_first_analyze, output_after_chatgpt_analyze_csv_username,overall_chain)


In [2]:
#This part use regular expression to analyze "appname" based on the stroage_path
request_evidence_type="appname"
combine_result_file_after_first_analyze = "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/5evidence_combinefiles_after_chatgpt_analyze_App_in_the_Document.csv"
output_after_chatgpt_analyze_csv_appname = "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/appname_after_chatgpt_analyze_App_in_the_Document.csv"


langchain_chatgpt.analyze_csv_file_appname(combine_result_file_after_first_analyze, output_after_chatgpt_analyze_csv_appname)

This stroage path does not has the regular package name /home/hzhou/llama2/Database_to_CSV/Database_to_CSV_Apps_in_the_Document/DatabaseToCSV__system_de__accounts_de.db/accounts.csv, its line number is 6.0
This app not found None, its line number is 6.0
This app not found stashes, its line number is 872.0
This app not found appmetadata, its line number is 991.0
This app not found stashes, its line number is 1048.0
This stroage path does not has the regular package name /home/hzhou/llama2/Database_to_CSV/Database_to_CSV_Apps_in_the_Document/DatabaseToCSV__app_si__content_store.db12345678910/blob_table.csv, its line number is 1151.0
This app not found None, its line number is 1151.0
This stroage path does not has the regular package name /home/hzhou/llama2/Database_to_CSV/Database_to_CSV_Apps_in_the_Document/DatabaseToCSV__app_si__content_store.db12345678910/blob_table.csv, its line number is 1152.0
This app not found None, its line number is 1152.0
This app not found stashes, its line n

In [3]:
#test the chain
overall_chain=langchain_chatgpt.Langchain_chatgpt_analyze("appname")
print(type(overall_chain.output_variables[0]))
text="""com.instagram.android"""
text2="""<string name="loggedInAccountUsername">thisisdfir100,  Sam Alex</string>"""
result=overall_chain(text)
test_string1 = result.get("appname","None")

result=overall_chain(text2)
test_string2 = result.get("name","None")

result_array= test_string1 + "\n" +test_string2
result_array

print(test_string1)
# type(result.get("name","None"))

<class 'str'>


> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.
Instagram


<font size="5">4. Transfer the csv file(after chatgpt analyze) to another csv file contain nodes and edges</font>

In [2]:
#Database file: Analyze sameline relationship and create grapgstep1
db_output_after_chatgpt_analyze_csv= "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/5evidence_combinefiles_after_chatgpt_analyze_App_in_the_Document.csv"
db_step1_output_file = "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/graphfiles/graphstep1_5evidence_combinefiles_Apps_in_the_Document.csv"
evidence_graph.db_transfer_graph_step1_sameline(db_output_after_chatgpt_analyze_csv, db_step1_output_file)

In [4]:
#Database file: Analyze sametable and samefolder relationship and create grapgstep2
db_step1_output_file = "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/graphfiles/graphstep1_5evidence_combinefiles_Apps_in_the_Document.csv"
db_step2_output_file = "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/graphfiles/graphstep2_5evidence_combinefiles_Apps_in_the_Document.csv"
evidence_graph.db_transfer_graph_step2_sametable_samedatabase(db_step1_output_file, db_step2_output_file)

KeyboardInterrupt: 

In [ ]:
# #Xml file: Analyze sametable and samefolder relationship and create grapgstep2
# xml__output_after_chatgpt_analyze_csv = "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/xml_after_chatgpt_analyze_App_in_the_Document.csv"
# xml_step1_output_file = "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/xml_graphstep1_Apps_in_the_Document.csv"
# evidence_graph.xml_transfer_graph_step1_sameline(xml__output_after_chatgpt_analyze_csv, xml_step1_output_file)

Transfer graph1 file to another file for analyze

In [4]:
db_step1_graph_file = "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/graphfiles/graphstep1_5evidence_combinefiles_Apps_in_the_Document.csv"
output_file_for_analyze = "/home/hzhou/llama2/llama/experiment_result/result/Apps_in_the_document/file_for_analyze_5evidence_combinefiles_Apps_in_the_Document.csv"
langchain_chatgpt.transfer_result_to_file_for_analyze(db_step1_graph_file, output_file_for_analyze)

{'node_evidence_number': '0.0.0', 'node_evidence': 'thisisdfir@gmail.com', 'evidence_type': 'email', 'edge_relationship_sameline': '[]', 'table_name': 'accounts.csv', 'row_number': '2', 'storage_path': '/home/hzhou/llama2/Database_to_CSV/Database_to_CSV_Apps_in_the_Document/DatabaseToCSV__com.google.android.googlequicksearchbox__opa_history/accounts.csv'}
{'node_evidence_number': '1.0.0', 'node_evidence': 'Josh Hickman', 'evidence_type': 'name', 'edge_relationship_sameline': '[]', 'table_name': 'entries.csv', 'row_number': '2', 'storage_path': '/home/hzhou/llama2/Database_to_CSV/Database_to_CSV_Apps_in_the_Document/DatabaseToCSV__com.google.android.googlequicksearchbox__opa_history/entries.csv'}
{'node_evidence_number': '2.0.0', 'node_evidence': 'Josh Hickman', 'evidence_type': 'name', 'edge_relationship_sameline': '[]', 'table_name': 'entries.csv', 'row_number': '6', 'storage_path': '/home/hzhou/llama2/Database_to_CSV/Database_to_CSV_Apps_in_the_Document/DatabaseToCSV__com.google.andr

In [8]:
#test
import os

file_path = '/path/to/your/file.txt'  # 用你的文件的实际路径替换这个

# 获取文件所在文件夹的名字
folder_name = os.path.basename(os.path.dirname(file_path))
current_folder = os.path.dirname(file_path)
parent_folder =  os.path.basename(os.path.dirname(current_folder))
# grandparent_folder = os.path.dirname(parent_folder)

print("文件所在文件夹的名字是:", folder_name)
print(current_folder)
print(parent_folder)
# print(grandparent_folder)

文件所在文件夹的名字是: your
/path/to/your
to



In [ ]:
import pandas as pd
pd.read_csv("/home/hzhou/llama2/llama/experiment_result/graphstep2.2.csv")

In [ ]:
#test
input_folder_csvfiles="/home/hzhou/llama2/Database_to_CSV_10apps/DatabaseToCSV__com.facebook.orca__threads_db2"
output_after_filter_csv="/home/hzhou/llama2/llama/after_filter/after_filter_facebooktest.csv"

preprocess_wordfilter.csv_extract_lines_with_keywords(input_folder_csvfiles,output_after_filter_csv) 

In [3]:
#test
path="/home/hzhou/llama2/Android10_Image/10 Sample apps/com.skout.android/shared_prefs/adref_sp.xml"
file_type = magic.from_file("/home/hzhou/llama2/Android10_Image/10 Sample apps/com.skout.android/shared_prefs/adref_sp.xml", mime=True)
tree = ET.parse(path)
root = tree.getroot()
def print_elements(element, indent=""):
    # 打印当前元素的信息
    print(f"{indent}Tag: {element.tag}, Attributes: {element.attrib}, Text: {element.text.strip() if element.text else 'None'}")
    # 递归遍历子元素
    for child in element:
        print_elements(child, indent + "    ")

# 开始遍历并打印信息
print_elements(root)

Tag: map, Attributes: {}, Text: 
    Tag: string, Attributes: {'name': 'adref'}, Text: ORGANIC
    Tag: string, Attributes: {'name': 'ref_str'}, Text: utm_source=google-play&utm_medium=organic


In [4]:
#test
# Define the regex pattern to match the app package name
import re
import pandas as pd
from google_play_scraper import app
file_path_1 = "/home/hzhou/llama2/Database_to_CSV/Database_to_CSV_Apps_in_the_Document/DatabaseToCSV__com.instagram.android__direct.db/messages.csv"
file_path_2 = "/home/hzhou/llama2/Database_to_CSV/Database_to_CSV_Apps_in_the_Document/DatabaseToCSV__comm.usica.abcv__downloader.db/downloader.csv"
pattern = r'__([a-zA-Z0-9.]+)__|/([a-zA-Z0-9]+\.[a-zA-Z0-9.]+)/'
match = re.search(pattern, file_path_1)
print(match)
if match:
    # Return the captured group (the app package name)
    package_name1=match.group(1) or match.group(2)
print(package_name1)


app_information = app(package_name1)
app_name=app_information.get("title", "Unknown App")

print(app_name)




<re.Match object; span=(85, 110), match='__com.instagram.android__'>
com.instagram.android
Instagram
